In [1]:
import os
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

import albumentations as A

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


apex is not installed


In [41]:
# Need to set
CONFIG_PATH = '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-16/mmsegmentation/configs/_TrashSEG_/swin/upernet_swin_large_patch4_window7_512x512_pretrain_224x224_22K_160k_ade20k.py'
TEST_IMAGES_PATH = '/opt/ml/input/data/test_images'
ITER = 'best_mIoU_epoch_58'
WORK_DIR = '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-16/mmsegmentation/work_dirs/upernet_swin_large_patch4_window7_512x512_pretrain_224x224_22K_160k_ade20k_0/'

In [42]:
cfg = Config.fromfile(CONFIG_PATH)
root=TEST_IMAGES_PATH

# dataset config 수정
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True
cfg.data.samples_per_gpu = 1
cfg.work_dir = WORK_DIR
# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{ITER}.pth')

In [43]:
dataset = build_dataset(cfg.data.test)
if len(dataset) != 819:
        raise AssertionError('Test dataset should 819 image. Check your test.json')
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2022-12-21 09:30:01,352 - mmseg - INFO - Loaded 819 images
/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-16/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: /opt/ml/input/level2_semanticsegmentation_cv-level2-cv-16/mmsegmentation/work_dirs/upernet_swin_large_patch4_window7_512x512_pretrain_224x224_22K_160k_ade20k_0/best_mIoU_epoch_58.pth


In [46]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 6.1 task/s, elapsed: 134s, ETA:     0s

In [50]:
# sample_submisson.csv 열기
submission = pd.read_csv('sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

# set resize
input_size = 512
output_size = 256
transformed = A.Compose([A.Resize(output_size, output_size)])
		

# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    temp_mask = []
    mask = np.array(predict, dtype='uint8')
    mask = transformed(image=mask)
    temp_mask.append(mask['image'])
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{ITER}.csv'), index=False)

print('Done')

Done


## Show image results

In [ ]:
from mmdet.apis.inference import init_segmentor
from mmdet.apis.inference import inference_segmentor
from mmdet.apis.inference import show_result_pyplot

In [ ]:
cfg = Config.fromfile('./configs/_trashDet_/dyhead/atss_swin-l-p4-w12_fpn_dyhead_mstrain_2x_coco_aug_3.py')
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

In [ ]:
model = init_segmentor(cfg, checkpoint=checkpoint_path, device='cuda:0')
# model.CLASSES = dataset.CLASSES

In [ ]:
imgs = []
imgs_idx = [0, 1, 2, 5, 11, 12] # , 11, 12 ,13 ,14, 15

for idx in imgs_idx:
    imgs.append(f'../../dataset/test/{idx:04d}.jpg')

In [ ]:
results = inference_segmentor(model, imgs)

for i in range(len(imgs)):
    show_result_pyplot(model, imgs[i], results[i])

## hard voting

In [1]:
import os
from os import path as osp
import pandas as pd
#import numpy as np
from tqdm import tqdm

In [2]:
base_dir = ''
target1 = 'submission_best_iter_80000.csv'
target2 = 'submission_best_mIoU_iter_52000.csv'
target3 = 'submission_iter_80000.csv'

df1 = pd.read_csv(osp.join(base_dir, target3))
df2 = pd.read_csv(osp.join(base_dir, target1))

submission = pd.DataFrame()
submission['image_id'] = df1['image_id']

In [3]:
CLASSES = [ 
    'Background', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass',
    'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'
]

In [5]:
PredictionString = []

for idx in tqdm(range(len(df1))):
    first = df1['PredictionString'][idx].split(' ')
    second = df2['PredictionString'][idx].split(' ')

    for i in range(len(first)):
        if CLASSES[int(second[i])] == 'Glass' or CLASSES[int(second[i])] == 'Paper pack':
            first[i] = second[i]
            
    result = ' '.join(first)
    PredictionString.append(result)

100%|██████████| 819/819 [00:31<00:00, 26.04it/s]


In [6]:
submission['PredictionString'] = PredictionString
submission.to_csv('./hard_voted_output.csv', index=False)